<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [299]:
import numpy as np 
import pandas as pd
import os, io
import datetime as dt
import pandas_datareader.data as web
import requests

# **TODO**


> Vergleiche mehrere Aktien miteinander



> Sende Push zu der Aktie, mit der besten Bewertung



> Prüfen wie stark der CCI gewachsen ist in den letzten 14 Tagen

# **Weitere Ideen**


> Aktien Sektoren zuordnen und den Bot nach Sektoren abfragen. Der Bot soll dann die stärkste Aktie aus dem Sektor zurückgeben. Ziemlich cool, wenn man weiß das zum Beispiel der Wasserstoffsektor gerade extrem am steigen ist.




> Sobald man sich für eine Aktie entschieden, wäre es ja auch interessant zu wissen, wann man am besten aussteigen sollte. Dann könnte man Abfragen schreiben und über den Bot eine Push rausschicken, ob man eine bestimmte Aktie, die man mit dem Bot teilt, verkaufen soll oder nicht, weil sich gerade der Kurs schlecht entwickelt.





# **Indikatoren**


> **MACD & Signallinie**: Der MACD (Moving Average Convergence/Divergence) wird in der Chartanalyse genutzt um Trends sowie Kauf-und Verkaufssignale zu erkennen. Dabei werden zwei exponentiell gewichtete gleitende Durchschnitte (26 und 12 Tage) berechnet und voneinander subtrahiert. Des Weiteren wird eine Signallinie berechnet. Hierbei handelt es sich um den 9 Tage exponentiell geglätteten Durchschnitt. Ein steigender MACD ist ein Indikator für einen Aufwärtstrend, ein sinkender MACD ein Indikator für einen Abwärtstrend. Schneidet der MACD die Signallinie von unten nach oben, so liegt ein Kaufsignal vor. Wird die Signallinie vom MACD von oben nach unten geschnitten, so liegt ein Verkaufssignal vor.



> **RSI**: Der RSI (Relative-Stärke-Index) ist ein Marktindikator. Er wird berechnet, indem der gleitende Mittelwert der Auf-und Abwärtsveränderungen für einen bestimmten Zeitraum (in der Regel 14 Tage) ermittelt wird. Der RSI kann Werte zwischen 0 und 100 einnehmen. Die Werte 30 und 70 spielen beim RSI eine besondere Rolle: Bei einem Wert unter 30 gilt der Wert als überverkauft. Dies bedeutet, dass eine Kurserholung (Steigerung) in nächster Zeit anstehen könnte. Bei einem Wert über 70 gilt die Aktie als überkauft. Dies ist ein Indiz dafür, dass der Kurs in nächster Zeit fallen könnte.




> **Bollinger Band**: Es existieren eigentlich zwei Bollinger Bänder, die gemeinsam mit dem gleitenden Mittel, dem arithmetischen Mittel der Kurse, dann das sogenannte Bollinger Band bilden. Mit Hilfe des Bollinger Bands werden der Verlauf der letzten Tage eines Wertpapieres und dessen Trend visualisiert. Man sieht, wo der Kurs durchschnittlich lag und wie stark er an welchen Tagen vom Mittelwert abwich. In der Regel verwendet man dafür den Zeitraum der letzten 20 Tage.





> **Bollinger Bands oder Keltner Channels?** Schau hier warum die BB die bessere Wahl wären: https://tradeciety.com/keltner-channel-vs-bollinger-bands-one-better/ Aber statt der Verwendung der Standardabweichung zur Darstellung der Bollinger Bänder wird die Average True Range von Wilders eingesetzt. Dieser Unterschied bewirkt, dass der Keltner Channel ruhiger und gleichmäßiger aussieht als die Bollinger Bänder.



> **EMA**: Exponential Moving Average verwendet man, um Kursschwankungen zu glätten und den zugrunde liegenden Trend dadurch besser zu erkennen. Hierzu betrachtet man einen Kursverlauf für einen bestimmten Zeitraum in der Vergangenheit und berechnet daraus den Durchschnitt. **Formel**: https://admiralmarkets.com/de/wissen/articles/forex-indicators/ema-der-exponentiell-gleitende-durchschnitt-als-universelles-hilfsmittel-fur-ihr-trading




> **Commodity Channel Index (CCI)**: Für den CCI wird die Abweichung zwischen Kurs und dessen Gleitenden Durchschnitt herangezogen, um das Momentum zu erfassen. Um dies zu bewerkstelligen, wird die Abweichung zwischen dem aktuellen Betrachtungszeitraum und der durchschnittlichen Abweichung der vergangenen X Zeitperioden verglichen. Im Gegensatz zum Stochastik-Oszillator und dem RSI sind die Werte des CCI nicht begrenzt. Aber in der Praxis bewegt sich der CCI aufgrund seiner Konstruktion zwischen -100 und +100. 
Bemerkenswerterweise werden beim CCI nicht die Schlusskurse, sondern die typischen Kurse (Hoch + Tief + Schlusskurs / 3) zugrunde gelegt. Dann wird die Abweichung bzw. Entfernung zwischen typischem Kurs und Gleitendem Durchschnitt ermittelt. **Formel**: https://www.kagels-trading.de/technische-indikatoren-trader/ **WICHTIG**: Wie der Name schon sagt, gründet der CCI auf der Annahme, dass sich die Kurse normalerweise innerhalb eines unsichtbaren Kanals um den Gleitenden Durchschnitt herum aufhalten.














# **Indikatoren mit denen wir anfangen sollten**


> 1. EMA


> 2. MACD


> 3. Bollinger Bänder



> 4. RSI












In [300]:
#indicator = pd.DataFrame(df, index=df.index)

In [301]:
def calc_indicators(df):
  df['ema_25'] = calc_ema_25(df)
  df['ema_100'] = calc_ema_100(df)

  df['sma_25'] = calc_sma_25(df)
  df['sma_100'] = calc_sma_100(df)

  calc_bollinger(df)

  calc_macd_signalline(df)

  df['RSI'] = calc_rsi(df)

  df['CCI'] = calc_cci(df)

  # df['OBV_MA'] = calc_obv(df)
   
  df = calculate_dead_cat_bounce_v2(df)

  return df

In [302]:
def calc_ema_25(df):
    ema_25 = 25
    df_ema_25 = df['Adj Close'].ewm(span=ema_25, adjust=False).mean()

    return df_ema_25

def calc_ema_100(df):
    ema_100 = 100
    df_ema_100 = df['Adj Close'].ewm(span=ema_100, adjust=False).mean()

    return df_ema_100

In [303]:
def calc_sma_25(df):
    sma_25 = 25
    df_sma_25 = df['Adj Close'].rolling(window=sma_25).mean()

    return df_sma_25

def calc_sma_100(df):
    sma_100 = 100
    df_sma_100 = df['Adj Close'].rolling(window=sma_100).mean()

    return df_sma_100

In [304]:
def calc_bollinger(df):
    bollinger_avg = 20

    short_avg_ma = df['Adj Close'].rolling(window=bollinger_avg).mean()
    short_avg_std = df['Adj Close'].rolling(window=bollinger_avg).std()

    df['bollinger'] = df['Adj Close'].ewm(span=bollinger_avg, adjust=False).mean()
    df['bollinger_lower_band'] = short_avg_ma - (short_avg_std *2)
    df['bollinger_upper_band'] = short_avg_ma + (short_avg_std *2)

In [305]:
def calc_macd_signalline(df):
    ema_short = 12
    ema_long = 26
    signalline_span = 9

    df_ema_short = df['Adj Close'].ewm(span=ema_short, adjust=False).mean()
    df_ema_long = df['Adj Close'].ewm(span=ema_long, adjust=False).mean()
    df['macd'] = df_ema_short - df_ema_long
    df['signalline'] = df['macd'].ewm(span=signalline_span, adjust=False).mean()

In [306]:
def calc_rsi(df, n=14):
    delta = df['Adj Close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    # RolUp = pd.rolling_mean(dUp, n)
    # RolDown = pd.rolling_mean(dDown, n).abs()

    # Calculate the EWMA
    RolUp = dUp.ewm(span=n).mean()
    RolDown = dDown.abs().ewm(span=n).mean()

    # Calculate the RSI based on EWMA
    RS = RolUp / RolDown
    RSI = 100 - (100/(1+RS))

    # Calculate the SMA
    roll_up2 = dUp.rolling(n).mean()
    roll_down2 = dDown.abs().rolling(n).mean()

    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))

    own_RSI=(RSI+RSI2)/2
    return own_RSI

In [307]:
def calc_cci(df, n=14, constant=0.015):
    tippingPrice = (df['High'] + df['Low'] + df['Adj Close']) / 3
    tippingPrice_mean = tippingPrice.rolling(n).mean()
    tippingPrice_std = tippingPrice.rolling(n).std()
    
    CCI = pd.Series((tippingPrice - tippingPrice_mean) / (constant * tippingPrice_std)) 
    return CCI

In [308]:
### To Be Adjusted

def calc_obv(df, n = 20):
    i = 0
    OBV = [0]

    while i < df.index[-1]:
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] > 0:
            OBV.append(df.loc[i + 1, 'Volume'])
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] == 0:
            OBV.append(0)
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] < 0:
            OBV.append(-df.loc[i + 1, 'Volume'])
        i = i + 1
    OBV = pd.Series(OBV)
    OBV_ma = pd.Series(OBV.rolling(n, min_periods=n).mean())

    return OBV_ma

In [309]:
def calculate_dead_cat_bounce_v2(df):
  set_freq = '2D'

  df['body_mid'] = (df['Adj Close'] + df['Open'])/2

  pct_change_close = df['body_mid'].pct_change(freq=set_freq, fill_method='ffill')
  pct_change_close.dropna(inplace=True)

  dead_cat_bounce = np.where((df.signalline > df.macd) &
                            ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                            ((df.Open < df.bollinger_lower_band)|(df.Close < df.bollinger_lower_band)),
                            True, False)

  df['dead_cat_bounce'] = dead_cat_bounce
  del df['body_mid']

  return df

In [310]:
def combine_stocks_to_dataframe(id_list):
  start = dt.datetime(2020,1,1)
  end = dt.datetime.now()
  #end = dt.datetime(2019,2,1)
  df_list = []

  for id in id_list:
    stock = web.DataReader(id, 'yahoo', start, end) # load data
    stock['stock_id'] = id
    stock = calc_indicators(stock) # calc indicators for every stock
    stock.dropna(inplace=True) # maybe delete this later
    df_list.append(stock)

  df_complete = pd.concat(df_list)
  
  return df_complete

In [311]:
stocks_dax = ['ADS.DE', 'ALV.DE', 'BAS.DE', 'BAYN.DE', 'BMW.DE', 'CON.DE', '1COV.DE', 'DAI.DE', 'DHER.DE', 'DBK.DE', 'DB1.DE', 'DPW.DE', 'DTE.DE', 'DWNI.DE', 'EOAN.DE', 'FRE.DE', 'FMS', 'HEI.DE', 'HEN3.DE', 'IFX.DE', 'LIN.DE', 'MRK', 'MTX.DE', 'MUV2.DE', 'RWE.DE', 'SAP.DE', 'SIE.DE', 'ENR.F', 'VOW.DE', 'VNA.DE']
stocks_random = ['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE', 'PLUG', 'MDO.DE', 'PYPL', 'V', 'YNDX']

stocks_combine = np.concatenate([stocks_dax, stocks_random])
df_complete = combine_stocks_to_dataframe(stocks_combine)

In [312]:
df_yd = df_complete.drop_duplicates(subset='stock_id', keep='last')

In [313]:
def compute_score_v2(row):
  score=0
  if row['ema_25']>row['ema_100']:
    score+=1
  if row['macd']>row['signalline']:
    score+=1
  if row['Adj Close']>row['bollinger_upper_band']:
    score-=1
  if ((row['RSI']>=50) & (row['RSI']<=70)) | (row['RSI'] <= 30):
    score+=1
  if row['Adj Close']<row['bollinger_lower_band']:
    score+=1
  if row['CCI']<-100:
      score+=1
  if row['Adj Close']>row['sma_25']:
      score+=1
  return score

In [314]:
def ema_score(row):
    ema_25 = row['ema_25']
    ema_100 = row['ema_100']
    score = (ema_25/ema_100)-1
    # if score < 0:
    #     score=0
    return score

def macd_signalline_score(row): # need to change something here
    # relative change might be distorted if values near by 0
    macd = row['macd']
    signalline = row['signalline']
    adj_close = row['Adj Close']
    score = (macd/adj_close+1)-(signalline/adj_close+1)
    return score

# set weighting when close is lower lower_band or upper upper_band
# to be discussed
def bollinger_score(row):
    lower_band = row['bollinger_lower_band']
    upper_band = row['bollinger_upper_band']
    middle_band = row['bollinger']
    adj_close = row['Adj Close']
    if (adj_close>=middle_band):
        if adj_close>=upper_band:
            score = ((adj_close/upper_band)-1)*(-1)
        else:
            score = ((adj_close/((upper_band-middle_band)*0.5+middle_band))-1)*(-1)
    else:
        if (adj_close<=lower_band):
            score = ((adj_close/lower_band)-1)*(-1)
        else:
            score = (adj_close/middle_band)-1
    return score

def sma_score(row):
    sma_25 = row['sma_25']
    adj_close = row['Adj Close']
    score = (adj_close/sma_25)-1
    return score

def rsi_score(row):
    lower_band = 30
    upper_band = 70
    middle_band = (lower_band + upper_band) / 2 
    middle_band1 = 55
    middle_band2 = 45
    outlier_factor = 1.2
    rsi = row['RSI']
    if (rsi>middle_band):
        if rsi>=upper_band:
            score = ((rsi/upper_band)-1)*(-1)*(outlier_factor)
        else:
            if rsi<middle_band1:
                score = 0
            else:
                if rsi<=62.5:
                    score = rsi/middle_band1-1
                else:
                    score = ((62.5-(rsi-62.5))/(middle_band1))-1
    else:
        if (rsi<=lower_band):
            score = ((rsi/lower_band)-1)*(-1)*(outlier_factor)
        else:
            if rsi>middle_band2:
                score = 0
            else:
                if rsi>=37.5:
                    score = (rsi/middle_band2-1)
                else:
                    score = (37.5-(rsi-37.5))/middle_band2-1
    return score

# We should check whether we give the same weighting if it hits above or lower 100/-100
def cci_score(row):
    lower_band = -100
    middle_lower_band = -50
    upper_band = 105
    middle_band = 0
    middle_upper_band = 85
    outlier_factor = 0.25
    outlier_factor1 = 2
    cci=row['RSI']

    if (cci < lower_band):
        score=(cci/lower_band)-1
    else:
        if (cci<=middle_lower_band):
            score = (cci / lower_band)-1
        elif (cci<middle_band):
            score = (((cci-lower_band)/lower_band)+1)*(-1)
        else:
            if (cci>upper_band):
                score = ((cci/upper_band)-1)*(-1)
            else:
                if(cci<=middle_upper_band):
                    score = (cci/upper_band)*outlier_factor
                else:
                    if(cci<upper_band):
                        score=(middle_upper_band-(cci-middle_upper_band))/upper_band-(middle_upper_band-(middle_upper_band-upper_band))/middle_upper_band
                    else:
                        score=0
    
    return score

In [315]:
def relative_score(row):
    r = row
    stock = row['stock_id']
    ema_scr = ema_score(r)
    macd_signalline_scr = macd_signalline_score(r)
    bollinger_scr = bollinger_score(r)
    sma_scr = sma_score(r)
    rsi_scr = rsi_score(r)
    cci_scr = cci_score(r)

    score = ema_scr + (macd_signalline_scr*5) + bollinger_scr + sma_scr + rsi_scr + cci_scr
    print(stock, ema_scr, (macd_signalline_scr*5), bollinger_scr, sma_scr, rsi_scr, cci_scr, score, '\n')
    return score

In [316]:
df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)
df_yd.sort_values(by=['score'], ascending=False)

ADS.DE 0.034595441694169615 -0.005176455584238182 0.01058796407388829 0.038201786238503344 0.09178763152719926 0.1429721898428475 0.3129685577923699 

ALV.DE 0.0542213912513283 -9.853806127835263e-05 -0.0052528311150656926 0.018851964414711153 0.06954352369209138 0.14005927095967863 0.2773247811414654 

BAS.DE 0.034285356185338234 0.0032811335553395438 0.011022414500104327 0.0045546096212865095 0 0.12658146669459938 0.179724980556668 

BAYN.DE 0.026971413453904303 -0.042042142393243775 -0.015692559807095163 -0.028893471973253315 -0.022035201351801592 0.07378948585912161 -0.007902476212367934 

BMW.DE 0.09560501011906042 0.05007315964962955 -0.0054510415098159015 0.11977490379555467 -0.2774823154203678 -0.43707000716631084 -0.4545502905322499 

CON.DE 0.03295679396567919 0.053318556939029316 -0.030199489968507187 0.09755508913980226 -0.21072694807857859 0.19593429834424703 0.13883830034167202 

1COV.DE 0.035820435652479166 0.013352845287970538 -0.00314325291278994 0.02208662733470601 0.

,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,sma_100,bollinger,bollinger_lower_band,bollinger_upper_band,macd,signalline,RSI,CCI,dead_cat_bounce,score
Date,,,,,,,,,,,,,,,,,,,,
2021-06-04,31.020000,30.370001,30.700001,30.889999,2038832.0,30.889999,RWE.DE,64.676546,121.487894,31.501600,131.524358,50.237378,30.196349,32.839651,-34.051710,-45.600306,41.401498,-82.025682,False,1.015754
2021-06-04,12.558000,12.314000,12.550000,12.374000,7375186.0,12.374000,DBK.DE,11.829362,10.606953,11.837200,10.375150,11.964707,11.196809,12.722391,0.397391,0.390624,63.819487,67.708162,False,0.425186
2021-06-04,17.326000,17.214001,17.271999,17.282000,4051080.0,17.282000,DTE.DE,16.858558,15.881987,16.809440,15.640719,16.943626,16.384827,17.525373,0.247165,0.243405,62.826476,70.852492,False,0.367948
2021-06-04,44.855000,44.500000,44.560001,44.825001,732005.0,44.825001,FRE.DE,43.062969,39.463993,43.163386,38.122576,43.511392,41.568135,46.173921,1.193265,1.373769,60.372854,17.635921,False,0.351385
2021-06-04,80.290001,78.970001,79.949997,79.540001,1748751.0,79.540001,DAI.DE,75.986879,69.486226,75.114800,69.423269,76.417403,70.712222,80.502779,1.468961,1.017504,69.562972,100.144090,False,0.340653
2021-06-04,58.680000,57.860001,57.860001,58.160000,653759.0,58.160000,1COV.DE,56.889374,54.922043,56.903200,56.517982,57.000897,55.097376,58.954625,0.396777,0.241456,61.809308,102.979542,False,0.339087
2021-06-04,312.000000,308.200012,310.000000,309.000000,42158.0,309.000000,VOW.DE,284.042044,248.993483,275.432001,243.047001,286.485415,241.929594,315.430406,9.448600,4.948683,79.799767,99.329418,False,0.330731
2021-06-04,299.250000,295.549988,298.000000,296.600006,325720.0,296.600006,ADS.DE,289.346584,279.671234,285.686280,279.176989,291.580040,278.835774,307.967965,6.814447,7.121514,60.048320,-10.664956,False,0.312969
2021-06-04,33.570000,32.900002,33.044998,33.380001,3217462.0,33.380001,IFX.DE,32.740348,32.676683,32.187600,33.939733,32.688498,30.226707,33.979293,0.019234,-0.256124,66.434578,70.480357,False,0.301860


In [317]:
# df_yd['score'] = df_yd.apply(lambda row: compute_score_v2(row), axis=1)
# df_yd

In [318]:
#df_complete[df_complete.dead_cat_bounce==True]

In [319]:
def bot_warn_me(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = '1790740607:AAE7Omfuea3tG3iii9A1qX-vQJ9pdLFcO0s'
    bot_chat_id = '908941210'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()